In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from sklearn.decomposition import PCA
from pyquaternion import Quaternion


In [3]:
# Load data
df_ref = pd.read_excel('Filtered_PCD_Annotations_AB.xlsx')

In [4]:
def get_object_perspective(cam_pose_x, cam_pose_y, box_x, box_y, box_z, box_rotation_w, box_rotation_z):
    """
    Determines the perspective of an object relative to the camera.

    Parameters:
    - box_center: np.array([x, y, z]) → center of the box in world coordinates
    - orientation_quat: Quaternion(w, 0, 0, z) → rotation around Z-axis only
    - cam_position: np.array([x, y, z]) → camera position in world coordinates
    - threshold: angle threshold in radians for classification

    Returns:
    - One of: 'toward', 'away', 'left', 'right'
    """
    threshold=np.pi / 4

    # Camera position in global coordinates
    cam_position = np.array([cam_pose_x, cam_pose_y, 0])  # shape: (3,)

    orientation_quat = Quaternion(
        box_rotation_w,
        0.0, 0.0,
        box_rotation_z
    )
    
    # Compute forward direction of the box (local +X axis rotated by orientation)
    forward_vector = orientation_quat.rotate(np.array([1, 0, 0]))  # shape: (3,)

    # Vector from box to camera
    box_center = [box_x, box_y, box_z]
    to_camera = cam_position - box_center
    to_camera = to_camera / np.linalg.norm(to_camera)

    # Compute angle between forward direction and camera vector
    dot = np.dot(forward_vector[:2], to_camera[:2])  # only XY plane
    angle = np.arccos(np.clip(dot, -1.0, 1.0))  # radians

    # Pad vectors to 3D by adding a zero Z-component
    forward_xy = np.array([forward_vector[0], forward_vector[1], 0])
    to_camera_xy = np.array([to_camera[0], to_camera[1], 0])
    
    # Compute cross product and extract Z-component
    cross_z = np.cross(forward_xy, to_camera_xy)[2]


    # Classify based on angle and cross product
    if angle < threshold:
        return 0 #'toward'
    elif angle > (np.pi - threshold):
        return 1 #'away'
    elif cross_z > 0:
        return 2 #'left'
    else:
        return 3 #'right'

In [5]:
def train_rf_model(X_train, X_test, y_train, y_test):
    scaler = StandardScaler()
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)

    param_grid = {
        'n_estimators': [100, 200],
        'max_depth': [None, 10, 20],
        'min_samples_split': [2, 5],
        'min_samples_leaf': [1, 2],
        'max_features': ['sqrt', 'log2']
    }

    rf = RandomForestRegressor(random_state=42)
    grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1)
    grid_search.fit(X_train_scaled, y_train)
    best_rf = grid_search.best_estimator_

    y_pred = best_rf.predict(X_test_scaled)
    metrics = {
        'RMSE': np.sqrt(mean_squared_error(y_test, y_pred)),
        'MAE': mean_absolute_error(y_test, y_pred),
        'R2': r2_score(y_test, y_pred)
    }

    return pd.Series(y_pred, index=y_test.index), metrics

In [ ]:

split_base = df_ref.dropna().copy()
train_idx, test_idx = train_test_split(split_base.index, test_size=0.2, random_state=42)
split_mask = pd.Series(False, index=df_ref.index)
split_mask.loc[train_idx] = True

df_x = df_ref[['a_image_coord_y', 'a_depth', 'a_radar_x']].dropna()
train_mask_x = split_mask.loc[df_x.index]
X_train_x = df_x.loc[train_mask_x, ['a_image_coord_y', 'a_depth']]
X_test_x = df_x.loc[~train_mask_x, ['a_image_coord_y', 'a_depth']]
y_train_x = df_x.loc[train_mask_x, 'a_radar_x']
y_test_x = df_x.loc[~train_mask_x, 'a_radar_x']
y_pred_x, metrics_x = train_rf_model(X_train_x, X_test_x, y_train_x, y_test_x)

# Step 4: Radar Y
df_y = df_ref[['a_image_coord_x', 'a_depth', 'a_radar_y']].dropna()
train_mask_y = split_mask.loc[df_y.index]
X_train_y = df_y.loc[train_mask_y, ['a_image_coord_x', 'a_depth']]
X_test_y = df_y.loc[~train_mask_y, ['a_image_coord_x', 'a_depth']]
y_train_y = df_y.loc[train_mask_y, 'a_radar_y']
y_test_y = df_y.loc[~train_mask_y, 'a_radar_y']
y_pred_y, metrics_y = train_rf_model(X_train_y, X_test_y, y_train_y, y_test_y)

# Step 5: VX/VY
df_v = df_ref.copy()

# Feature engineering
le = LabelEncoder()
df_v['a_category_encoded'] = le.fit_transform(df_v['a_category'])

df_v['a_box_perspective'] = df_v.apply(
    lambda row: get_object_perspective(
        row['a_camera_pose_x'], row['a_camera_pose_y'],
        row['a_center_x'], row['a_center_y'], row['a_center_z'],
        row['a_rotation_w'], row['a_rotation_z']
    ), axis=1
)

df_v = df_v.drop_duplicates()

# Translation deltas
df_v['delta_center_x'] = df_v['b_center_x'] - df_v['a_center_x']
df_v['delta_center_y'] = df_v['b_center_y'] - df_v['a_center_y']
df_v['delta_center_z'] = df_v['b_center_z'] - df_v['a_center_z']
df_v['delta_length'] = df_v['b_length'] - df_v['a_length']
df_v['delta_width'] = df_v['b_width'] - df_v['a_width']
df_v['delta_height'] = df_v['b_height'] - df_v['a_height']
df_v['delta_vehicle_speed'] = df_v['b_vehicle_speed'] - df_v['a_vehicle_speed']

def compute_delta_quaternion(row):
    q1 = Quaternion([row['a_rotation_w'], 0.0, 0.0, row['a_rotation_z']])
    q2 = Quaternion([row['b_rotation_w'], 0.0, 0.0, row['b_rotation_z']])
    delta_q = q2 * q1.inverse
    return pd.Series({
        'delta_rotation_w': delta_q.w,
        'delta_rotation_x': delta_q.x,
        'delta_rotation_y': delta_q.y,
        'delta_rotation_z': delta_q.z
    })

df_quat = df_v.apply(compute_delta_quaternion, axis=1)
df_v = pd.concat([df_v, df_quat], axis=1)

df_v = df_v.loc[:, df_v.nunique() > 1]
df_v = df_v.drop(['a_category', 'b_category'], axis=1)
df_v = pd.get_dummies(df_v, drop_first=True)

# Prepare features
numeric_df = df_v.select_dtypes(include='number').drop(['a_vx', 'a_vy', 'b_vx', 'b_vy'], axis=1)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(numeric_df)
pca = PCA(n_components=min(10, X_scaled.shape[1]))
X_pca = pca.fit_transform(X_scaled)

y_vx = df_v['a_vx']
y_vy = df_v['a_vy']
train_mask_v = split_mask.loc[df_v.index]
X_train_pca = X_pca[train_mask_v.values]
X_test_pca = X_pca[~train_mask_v.values]
y_train_vx = y_vx.loc[train_mask_v]
y_test_vx = y_vx.loc[~train_mask_v]
y_train_vy = y_vy.loc[train_mask_v]
y_test_vy = y_vy.loc[~train_mask_v]

y_pred_vx, metrics_vx = train_rf_model(X_train_pca, X_test_pca, y_train_vx, y_test_vx)
y_pred_vy, metrics_vy = train_rf_model(X_train_pca, X_test_pca, y_train_vy, y_test_vy)

# Step 6: Consolidated output
df_results = pd.DataFrame({
    'Radar_X_True': y_test_x,
    'Radar_X_Pred': y_pred_x,
    'Radar_Y_True': y_test_y,
    'Radar_Y_Pred': y_pred_y,
    'VX_True': y_test_vx,
    'VX_Pred': y_pred_vx,
    'VY_True': y_test_vy,
    'VY_Pred': y_pred_vy
})

metrics_summary = pd.DataFrame({
    'Target': ['Radar_X', 'Radar_Y', 'VX', 'VY'],
    'RMSE': [metrics_x['RMSE'], metrics_y['RMSE'], metrics_vx['RMSE'], metrics_vy['RMSE']],
    'MAE': [metrics_x['MAE'], metrics_y['MAE'], metrics_vx['MAE'], metrics_vy['MAE']],
    'R2': [metrics_x['R2'], metrics_y['R2'], metrics_vx['R2'], metrics_vy['R2']]
})

print("📊 Model Evaluation Summary:")
print(metrics_summary.round(4))


📊 Model Evaluation Summary:
    Target    RMSE     MAE      R2
0  Radar_X  2.1075  0.8245  0.9813
1  Radar_Y  2.3987  1.4498  0.9054
2       VX  1.0760  0.4561  0.9236
3       VY  0.6214  0.2806  0.9381


In [17]:
y_pred_y.isna().sum()

np.int64(0)

In [8]:
df_results.sample(20)

,Radar_X_True,Radar_X_Pred,Radar_Y_True,Radar_Y_Pred,VX_True,VX_Pred,VY_True,VY_Pred
3827,10.400000,10.459267,NaN,NaN,-6.00,-5.998750,3.25,3.477507
4429,12.800000,13.266738,NaN,NaN,-5.00,-5.089238,1.25,1.523571
4054,19.400000,19.612624,-0.3,-1.039781,-4.25,-3.781938,5.25,5.575505
2800,25.000000,27.012624,NaN,NaN,-5.00,-4.828720,0.00,0.012768
414,29.200001,26.312739,10.9,7.652769,-6.00,-6.935885,-1.00,-0.275373
2136,47.599998,48.094568,1.1,1.016408,0.00,-0.008958,0.00,-0.004020
3498,16.600000,16.726916,NaN,NaN,-4.00,-4.211856,6.75,6.661628
3672,13.600000,13.589469,NaN,NaN,-5.50,-5.273065,4.75,4.825381
842,13.600000,13.008184,NaN,NaN,-2.00,-2.382414,-1.25,-1.245390
1799,17.600000,17.617364,NaN,NaN,-2.00,-1.785423,0.00,0.002490
